# Limpieza de datos

In [1]:
import pandas as pd
import re   

In [2]:
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0-1'
#FILE='CIS_Microsoft_Intune_for_Windows_10_Benchmark_v2.0.0'
FILE='CIS_Microsoft_Windows_Server_2008_(non-R2)_Benchmark_v3.3.1_ARCHIVE'
#FILE='CIS_Microsoft_Windows_Server_2012_(non-R2)_Benchmark_v3.0.0_FINAL_UPDATE'
#FILE='CIS_Microsoft_Windows_Server_2016_Benchmark_v2.0.0'
#FILE='CIS_Microsoft_Windows_Server_2016_STIG_Benchmark_v2.0.0' 

In [3]:
df=pd.read_excel('../../output/Windows/extracted/'+FILE+'_extracted.xlsx')

In [4]:
df

,IDCIS,Level,Title,Description,Audit,Remediation
0,1.1.1,(L1),Ensure 'Enforce password history' is set to '...,This policy setting determines the number of r...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
1,1.1.2,(L1),Ensure 'Maximum password age' is set to '365 ...,This policy setting defines how long a user ca...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
2,1.1.3,(L1),Ensure 'Minimum password age' is set to '1 or...,This policy setting determines the number of d...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
3,1.1.4,(L1),Ensure 'Minimum password length' is set to '1...,This policy setting determines the least numbe...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
4,1.1.5,(L1),Ensure 'Password must meet complexity require...,This policy setting checks all new passwords t...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
...,...,...,...,...,...,...
312,19.7.4.1,(L1),Ensure 'Do not preserve zone information in f...,This policy setting allows you to manage wheth...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
313,19.7.4.2,(L1),Ensure 'Notify antivirus programs when openin...,This policy setting manages the behavior for n...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
314,19.7.28.1,(L1),Ensure 'Prevent users from sharing files with...,This policy setting determines whether users c...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...
315,19.7.43.1,(L1),Ensure 'Always install with elevated privileg...,This setting controls whether or not Windows I...,Navigate to the UI Path articulated in the Rem...,To establish the recommended configuration via...


A continuación se realiza la limpieza a cada una de las columnas para ello se consideran eliminar caracteres especiales como espacions en blanco, saltos de linea, tabulaciones, etc. Eliminar cadenas de caracteres que no aportan información relevante para el análisis, y además 

In [6]:
df['Title'][13]

" Ensure 'Add workstations to domain' is set to \n'Administrators' (DC only) (Automated) \n"

### Title

In [5]:
df['Title']=df['Title'].str.replace('(Automated)','')
df['Title']=df['Title'].str.strip()

### Nueva columna MS/DC

In [6]:
PATTERN_EXTRACT_ADD=re.compile(r'\((MS only|DC only)\)',re.DOTALL)
df['MS/DC']=df['Title'].apply(
    lambda x: 
        PATTERN_EXTRACT_ADD.findall(x)[0] if len(PATTERN_EXTRACT_ADD.findall(x))>0 else 'None'
)

In [7]:
df['Title']=df['Title'].str.replace('(MS only)','')
df['Title']=df['Title'].str.replace('(DC only)','')

### Audit

In [8]:
df['Audit']=df['Audit'].apply(lambda x: re.compile(r'Page \d+', re.DOTALL).sub('',x))
df['Audit']=df['Audit'].str.strip()

### Remediation

In [9]:
PATTERN_REMEDIATION= re.compile(r'(.*?)(\nNote|$)', re.DOTALL)

In [10]:
df['Remediation']=df['Remediation'].apply(lambda x: PATTERN_REMEDIATION.findall(x)[0][0])
df['Remediation']=df['Remediation'].apply(lambda x: re.compile(r'Page \d+', re.DOTALL).sub('',x))
df['Remediation']=df['Remediation'].str.strip()

#### Extraer Recommended state

Dentro de la columna de **description** se encuentra el estado recomendado para cada una de los controles, este debe ir en la columna de **Procedimiento de implementación**,
por lo que se extrae en una nueva columna que se nombrara **State**.


In [11]:
PATTERN_STATE=re.compile(r'(\nThe recommended .*?|$)(Note|Important|$)', re.DOTALL)

In [12]:
df['State']=df['Description'].apply(lambda x: PATTERN_STATE.search(x).group(1))
df['State']=df['State'].apply(lambda x: re.compile(r'Page \d+', re.DOTALL).sub('',x))
df['State']=df['State'].str.strip()

### Description

Una vez se ha extraido se limita la descripción hasta antes de la sección de estado recomendo o en algunos casos cuando empiezan las notas adicionales de la descripción

In [13]:
PATTERN_DESCRIPTION=re.compile(r'(.*?)(\nThe recommended|Note|$)', re.DOTALL)

In [14]:
df['Description']=df['Description'].apply(lambda x: re.compile(r'Page \d+', re.DOTALL).sub('',x))
df['Description']=df['Description'].apply(lambda x: PATTERN_DESCRIPTION.search(x).group(1))
df['Description']=df['Description'].str.strip()

In [15]:
DF=df

# Dataframe final

In [16]:
DF_final=pd.DataFrame(columns=["IDCIS", "Parametro", "MS/DS", "Descripción", "Procedimiento de implementación", "Procedimiento de verificación", "Config implementación", "Config verificación"])

In [17]:
DF_final['IDCIS']=DF['IDCIS']
DF_final['Parametro']=DF['Title']
DF_final['MS/DS']=DF['MS/DC']
DF_final['Descripción']=DF['Description']

## Dividir zonas sin traducción

Debido a que tanto implementación como configuración contienen secciones de configuración que no desean ser traducidas se extraen cada una de estas secciones y se almacenan en columnas separadas, para despues unirlas una vez se haya realizado la traducción.

In [18]:
PATTERN_DIV_REMEDIATION=re.compile(r'(: \n)', re.DOTALL)
"".join(PATTERN_DIV_REMEDIATION.split(DF['Remediation'][0])[:-1])

'To establish the recommended configuration via GP, set the following UI path to 24 or \nmore password(s): \n'

In [19]:
DF_final['Procedimiento de implementación']=DF['Remediation'].apply(lambda x: "".join(PATTERN_DIV_REMEDIATION.split(x)[:-1]))
DF_final['Config implementación']=DF['Remediation'].apply(lambda x: PATTERN_DIV_REMEDIATION.split(x)[-1])

In [20]:
PATTERN_DIV_AUDIT=re.compile(r'(: \n)', re.DOTALL)
PATTERN_DIV_AUDIT.split(DF['Audit'][130])

['Navigate to the UI Path articulated in the Remediation section and confirm it is set as \nprescribed. This group policy setting is backed by the following registry location',
 ': \n',
 'HKEY_LOCAL_MACHINE\\SOFTWARE\\Policies\\Microsoft\\WindowsFirewall\\PrivateProfile\n:DefaultOutboundAction']

In [21]:
DF_final['Procedimiento de verificación']=DF['Audit'].apply(lambda x: "".join(PATTERN_DIV_AUDIT.split(x)[:-1]))
DF_final['Procedimiento de verificación']=DF['Audit'].apply(
    lambda x:
    "".join(PATTERN_DIV_AUDIT.split(x)[:-1]) if len(PATTERN_DIV_AUDIT.split(x))>1 
    else x
)


In [22]:
DF_final['Config verificación']=DF['Audit'].apply(
    lambda x:
    PATTERN_DIV_AUDIT.split(x)[-1] if len(PATTERN_DIV_AUDIT.split(x))>1
    else ""
)


### Procedimiento de implementación
Esta columna esta compuesta por la sección de state y la sección de remmediation, por lo que se procede a unir en esta ambas columnas

In [23]:
DF_final['Procedimiento de implementación']=DF['State']+" "+DF_final['Procedimiento de implementación']

In [24]:
DF_final

,IDCIS,Parametro,MS/DS,Descripción,Procedimiento de implementación,Procedimiento de verificación,Config implementación,Config verificación
0,1.1.1,Ensure 'Enforce password history' is set to '2...,None,This policy setting determines the number of r...,The recommended state for this setting is: 24 ...,Navigate to the UI Path articulated in the Rem...,Computer Configuration\Policies\Windows Settin...,
1,1.1.2,Ensure 'Maximum password age' is set to '365 o...,None,This policy setting defines how long a user ca...,The recommended state for this setting is 365 ...,Navigate to the UI Path articulated in the Rem...,Computer Configuration\Policies\Windows Settin...,
2,1.1.3,Ensure 'Minimum password age' is set to '1 or ...,None,This policy setting determines the number of d...,The recommended state for this setting is: 1 o...,Navigate to the UI Path articulated in the Rem...,Computer Configuration\Policies\Windows Settin...,
3,1.1.4,Ensure 'Minimum password length' is set to '14...,None,This policy setting determines the least numbe...,The recommended state for this setting is: 14 ...,Navigate to the UI Path articulated in the Rem...,Computer Configuration\Policies\Windows Settin...,
4,1.1.5,Ensure 'Password must meet complexity requirem...,None,This policy setting checks all new passwords t...,The recommended state for this setting is: Ena...,Navigate to the UI Path articulated in the Rem...,Computer Configuration\Policies\Windows Settin...,
...,...,...,...,...,...,...,...,...
312,19.7.4.1,Ensure 'Do not preserve zone information in fi...,None,This policy setting allows you to manage wheth...,The recommended state for this setting is: Dis...,Navigate to the UI Path articulated in the Rem...,User Configuration\Policies\Administrative Tem...,HKEY_USERS\[USER \nSID]\Software\Microsoft\Win...
313,19.7.4.2,Ensure 'Notify antivirus programs when opening...,None,This policy setting manages the behavior for n...,The recommended state for this setting is: Ena...,Navigate to the UI Path articulated in the Rem...,User Configuration\Policies\Administrative Tem...,HKEY_USERS\[USER \nSID]\Software\Microsoft\Win...
314,19.7.28.1,Ensure 'Prevent users from sharing files withi...,None,This policy setting determines whether users c...,The recommended state for this setting is: Ena...,Navigate to the UI Path articulated in the Rem...,User Configuration\Policies\Administrative Tem...,HKEY_USERS\[USER \nSID]\Software\Microsoft\Win...
315,19.7.43.1,Ensure 'Always install with elevated privilege...,None,This setting controls whether or not Windows I...,The recommended state for this setting is: Dis...,Navigate to the UI Path articulated in the Rem...,User Configuration\Policies\Administrative Tem...,HKEY_USERS\[USER \nSID]\Software\Policies\Micr...


In [25]:
DF_final.replace('\n','', regex=True, inplace=True)

In [26]:
DF_final.to_excel('../../output/Windows/clean/'+FILE+'_clean'+'.xlsx', index=False)